# Steganography

The goal is to hide a secret message in a png. Steps we need to take:

1. Load an image as an array of bytes
2. Convert message into bits
3. For each bit in message, change the lsb of every pixel until the entire message is encoded

*Optional:* 
- Ensure the image is large enough to contain the message. 
- Choose how many bits to change per pixel
  - Distribute bit change equally on all channels

That's it for **encoding** the message, but we also need to **decode** it. To do that, we need to know the size of the message, so we need to make sure the message contains some header. 

This is the image we will be working on:

<img src="nomorefanmail.png" width="500">

First we need to **encode the message in binary**:

In [201]:
def str_to_bin(str : str) -> str:
    # Different approaches to achieve the same thing: 
    # bits = [format(ord(char), '08b') for char in msg]                 # eg. ['01010111', '01100001', '01110010', '00100000']
    # from https://www.geeksforgeeks.org/python/python-convert-string-to-binary/ :
    # bits = ''.join(format(ord(char), '08b') for char in msg)          # eg. "01010111011000010111001000100000011000010110111001"
    bits = [bit for char in str for bit in format(ord(char), '08b')]    # eg. ['0', '1', '0', '1', '0', '1', '1', '1', '0']
    return bits

def bin_to_str(bits : list) -> str:
    # for every 8 elements in bits, join 8 bits to a "bitstring" and convert them to a unicode character
    chars = [chr(int(''.join(bits[i:i+8]), 2)) for i in range(0, len(bits), 8)]
    original_msg = ''.join(chars)
    return original_msg

Then, we'll **load the image**:

In [202]:
from PIL import Image
import numpy as np

def load_image(path : str):
    img = Image.open(path) 
    pixels = np.array(img, dtype=np.uint8) # this gives us a 2D array, pixels[0][0] gives us the data of the first pixel
    # example pixel: [224 238 241]
    return pixels

path = "nomorefanmail.png"
pixels = load_image(path)

We'll define a function that takes a pixel, and **hides one bit in one of the channels**.

In [203]:
def encode_on_pixel(pixel, msg_bit: str):
    # side effects: (none)
    
    if len(msg_bit) != 1:
        raise ValueError(f"Error: Invalid bit length: {len(msg_bit)}")
    
    if msg_bit != '1' and msg_bit != '0':
        raise ValueError(f"Error: Invalid message bit: {msg_bit}")
    
    red_channel, green_channel, blue_channel = pixel

    binary = list(f'{blue_channel:08b}') # Assuming the channel consists of 8 bits (which is standard in most formats)
    binary[-1] = msg_bit

    new_pixel = red_channel, green_channel, np.uint8(int(''.join(binary), 2))

    return new_pixel
    

Now we'll call this function on all pixels, until the message is encoded.

In [204]:
def encode_message(pixels, msg):
    # WARNING FOR SIDE EFFECTS: 'pixels' will be altered direcly
    
    msg_with_len = str(len(msg)) + "||" + msg
    encoded_msg = str_to_bin(msg_with_len)

    bit_index = 0

    for i, row in enumerate(pixels):
        for j, pixel in enumerate(row):
            bit = encoded_msg[bit_index]
            # print(f"before: {pixel}")
            pixels[i][j] = encode_on_pixel(pixel, bit)
            # print(f"after: {pixel}")
            bit_index += 1
            if bit_index >= len(encoded_msg): break
        if bit_index >= len(encoded_msg): break

    return pixels

After encoding the message, we just need to **store** it:

In [205]:
def store_image(pixels, file_name : str):
    # side effects: saves image

    image = Image.fromarray(pixels, 'RGB')
    image.save(file_name)

But of course, we also need to be able to **decode the message**. Otherwise, all this work was for nothing. 

In [206]:
import re


def decode_pixel(pixel):
    red_channel, green_channel, blue_channel = pixel
    binary = list(f'{blue_channel:08b}') # Assuming the channel consists of 8 bits (which is standard in most formats)
    return binary[-1]

def decode_message(pixels):
    bits = []

    for row in pixels:
        for pixel in row:
            bits.append(decode_pixel(pixel))

    decoded = bin_to_str(bits)

    msg_size_str, msg = re.split(r"\|\|", decoded, maxsplit=1)
    msg_size = int(msg_size_str)
    
    return msg[:msg_size]

Now, we can test the code:

In [207]:
msg = "War and hate, war and hate!"

pixels1 = load_image("nomorefanmail.png")
encode_message(pixels1, msg)
store_image(pixels1, "secret.png") ## You can't store in jpg, since it's a lossy format. The message will break!

decoded1 = decode_message(pixels1)

print(f"secret message:           {msg}")
print(f"decoded from image:       {decoded1}")

pixels2 = load_image("secret.png") 
decoded2 = decode_message(pixels2)
print(f"decoded from saved image: {decoded2}")

secret message:           War and hate, war and hate!
decoded from image:       War and hate, war and hate!
decoded from saved image: War and hate, war and hate!


You probably can't even tell the difference between the original and the modifed version:

| Original | Modified |
|----------|----------|
| <img src="nomorefanmail.png" width="70%">| <img src="secret.png" width="70%">|